In [ ]:
import pandas as pd

# 파일 경로 지정
file_path = '/content/drive/MyDrive/osp_team_project/'
file_name = 'data_sports_notext.csv'

# CSV 파일을 데이터프레임으로 읽기
df = pd.read_csv(file_path + file_name)
df

In [ ]:
# NaN 값을 '.'으로 채우기
df['Description'] = df['Description'].fillna('.')

In [ ]:
! pip install youtube-transcript-api

In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi

# YouTube 동영상 ID 추출 함수
def extract_video_id(link):
    return link.split('v=')[1]

# 각 행에 대해 YouTube 동영상 ID 추출 및 자막 가져오기
for index, row in df.iterrows():
    video_id = extract_video_id(row['Video Link'])

    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=['ko'])
    except Exception as e:
        print(f"Error fetching transcript for video {video_id}: {e}")
        transcript = []

    # 자막이 존재하면 텍스트로 변환하여 'Transcript' 열에 추가
    if transcript:

      text = ''

      for i in range(len(transcript)):
        text += transcript[i]['text'] + ' ' # text 부분만 가져옴
        df.at[index, 'Transcript'] = text

    else:
        df.at[index, 'Transcript'] = None

In [ ]:
df

In [ ]:
df = df.dropna(subset=['Transcript'])
df

In [ ]:
df.info()

In [ ]:
!pip install transformers
!pip install sentence_transformers

In [ ]:
from transformers import T5ForConditionalGeneration,AutoTokenizer
path = "lcw99/t5-large-korean-text-summary"
model = T5ForConditionalGeneration.from_pretrained(path)
tokenizer = AutoTokenizer.from_pretrained(path)

In [ ]:
df_sum = df.copy()

In [ ]:
# 데이터프레임의 각 행에 대해 summary 생성 및 추가
for index, row in df_sum.iterrows():
    text = row['Transcript']
    print(len(text))
    # 텍스트를 토큰화하고 모델로 summary 생성
    token = tokenizer(text[:5000], return_tensors="pt")
    output = model.generate(input_ids=token["input_ids"], attention_mask=token["attention_mask"], max_length=150)
    summary = tokenizer.decode(output[0])[5:-4]
    print(index)

    # 생성된 summary를 데이터프레임에 추가
    df_sum.at[index, 'Summary'] = summary

In [ ]:
# 'Transcript' 열을 DataFrame에서 제거
df_sum = df_sum.drop(columns=['Transcript'])

# 결과 확인
df_sum


In [ ]:
dfdf = df_sum.copy()
def remove_consecutive_duplicates_in_summary(row):
    # 'Summary' 열의 값을 공백 단위로 나눔
    words = row['Summary'].split(' ')

    # 중복을 제거한 단어 리스트 생성
    unique_words = [words[0]]
    for word in words[1:]:
        if word != unique_words[-1]:
            unique_words.append(word)

    # 리스트를 다시 문자열로 결합하여 반환
    return ' '.join(unique_words)

# 'Summary' 열의 각 행에 대해 중복된 어구 제거 함수 적용
dfdf['Summary'] = dfdf.apply(remove_consecutive_duplicates_in_summary, axis=1)

In [ ]:
save_path = "/content/drive/MyDrive/osp_team_project/youtube_data/"
save_name = "data_sports.csv"

dfdf.to_csv(save_path + save_name, index=False)

In [ ]:
# '#shorts'가 포함된 행과 포함되지 않은 행으로 데이터프레임 분리
df_shorts = df[df['Description'].str.contains('#shorts')]
df_normal = df[~df['Description'].str.contains('#shorts')]

In [ ]:
df_shorts.head()